## TF IDF con PySpark

In [1]:
import pyspark
import math

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

pyspark.context.SparkContext

In [83]:
# Leo el CSV de textos a indexar
textosRDD = sc.textFile('./textos.csv')

# Separo por comas
textosRDD = textosRDD.map(lambda a: a.split(','))
textosRDD.collect()

[['1', 'VW VW FORD FIAT'],
 ['2', 'ALFA ROMEO LANCIA ALFA ROMEO'],
 ['3', 'FERRARI FIAT ALFA ROMEO LANCIA'],
 ['4', 'FORD FORD FIAT CHEVROLET FIAT'],
 ['5', 'CHEVROLET VW CHEVROLET VW VW'],
 ['6', 'FIAT FERRARI FERRARI']]

In [63]:
def tokenizar(x):
    l = x[1].split()
    for i in range(len(l)):
        l[i] = (l[i], x[0])
    return l

In [84]:
textosRDD = textosRDD.flatMap(lambda x: tokenizar(x))
textosRDD.collect()

[('VW', '1'),
 ('VW', '1'),
 ('FORD', '1'),
 ('FIAT', '1'),
 ('ALFA', '2'),
 ('ROMEO', '2'),
 ('LANCIA', '2'),
 ('ALFA', '2'),
 ('ROMEO', '2'),
 ('FERRARI', '3'),
 ('FIAT', '3'),
 ('ALFA', '3'),
 ('ROMEO', '3'),
 ('LANCIA', '3'),
 ('FORD', '4'),
 ('FORD', '4'),
 ('FIAT', '4'),
 ('CHEVROLET', '4'),
 ('FIAT', '4'),
 ('CHEVROLET', '5'),
 ('VW', '5'),
 ('CHEVROLET', '5'),
 ('VW', '5'),
 ('VW', '5'),
 ('FIAT', '6'),
 ('FERRARI', '6'),
 ('FERRARI', '6')]

In [85]:
textosRDD = textosRDD.map(lambda x: ((x[0], x[1]),1))
textosRDD.collect()

[(('VW', '1'), 1),
 (('VW', '1'), 1),
 (('FORD', '1'), 1),
 (('FIAT', '1'), 1),
 (('ALFA', '2'), 1),
 (('ROMEO', '2'), 1),
 (('LANCIA', '2'), 1),
 (('ALFA', '2'), 1),
 (('ROMEO', '2'), 1),
 (('FERRARI', '3'), 1),
 (('FIAT', '3'), 1),
 (('ALFA', '3'), 1),
 (('ROMEO', '3'), 1),
 (('LANCIA', '3'), 1),
 (('FORD', '4'), 1),
 (('FORD', '4'), 1),
 (('FIAT', '4'), 1),
 (('CHEVROLET', '4'), 1),
 (('FIAT', '4'), 1),
 (('CHEVROLET', '5'), 1),
 (('VW', '5'), 1),
 (('CHEVROLET', '5'), 1),
 (('VW', '5'), 1),
 (('VW', '5'), 1),
 (('FIAT', '6'), 1),
 (('FERRARI', '6'), 1),
 (('FERRARI', '6'), 1)]

In [86]:
textosRDD = textosRDD.reduceByKey(lambda x,y: x + y)
textosRDD.collect()

[(('ALFA', '2'), 2),
 (('ROMEO', '2'), 2),
 (('FIAT', '3'), 1),
 (('ALFA', '3'), 1),
 (('ROMEO', '3'), 1),
 (('CHEVROLET', '5'), 2),
 (('VW', '5'), 3),
 (('FIAT', '6'), 1),
 (('VW', '1'), 2),
 (('FORD', '1'), 1),
 (('FIAT', '1'), 1),
 (('LANCIA', '2'), 1),
 (('FERRARI', '3'), 1),
 (('LANCIA', '3'), 1),
 (('FORD', '4'), 2),
 (('FIAT', '4'), 2),
 (('CHEVROLET', '4'), 1),
 (('FERRARI', '6'), 2)]

In [88]:
palabras = textosRDD.map(lambda x: (x[0][0],1))
palabras.collect()

[('ALFA', 1),
 ('ROMEO', 1),
 ('FIAT', 1),
 ('ALFA', 1),
 ('ROMEO', 1),
 ('CHEVROLET', 1),
 ('VW', 1),
 ('FIAT', 1),
 ('VW', 1),
 ('FORD', 1),
 ('FIAT', 1),
 ('LANCIA', 1),
 ('FERRARI', 1),
 ('LANCIA', 1),
 ('FORD', 1),
 ('FIAT', 1),
 ('CHEVROLET', 1),
 ('FERRARI', 1)]

In [89]:
palabras = palabras.reduceByKey(lambda x,y: x + y)
palabras.collect()

[('LANCIA', 2),
 ('FERRARI', 2),
 ('ALFA', 2),
 ('ROMEO', 2),
 ('FIAT', 4),
 ('CHEVROLET', 2),
 ('VW', 2),
 ('FORD', 2)]

In [96]:
idf = palabras.map(lambda x: (x[0],math.log10(7/x[1])))
idf.collect()

[('LANCIA', 0.5440680443502757),
 ('FERRARI', 0.5440680443502757),
 ('ALFA', 0.5440680443502757),
 ('ROMEO', 0.5440680443502757),
 ('FIAT', 0.24303804868629444),
 ('CHEVROLET', 0.5440680443502757),
 ('VW', 0.5440680443502757),
 ('FORD', 0.5440680443502757)]

In [80]:
textosRDD = textosRDD.map(lambda x: (x[0][1],(x[0][0],x[1])))
textosRDD.collect()

[('2', ('ALFA', 2)),
 ('2', ('ROMEO', 2)),
 ('3', ('FIAT', 1)),
 ('3', ('ALFA', 1)),
 ('3', ('ROMEO', 1)),
 ('5', ('CHEVROLET', 2)),
 ('5', ('VW', 3)),
 ('6', ('FIAT', 1)),
 ('1', ('VW', 2)),
 ('1', ('FORD', 1)),
 ('1', ('FIAT', 1)),
 ('2', ('LANCIA', 1)),
 ('3', ('FERRARI', 1)),
 ('3', ('LANCIA', 1)),
 ('4', ('FORD', 2)),
 ('4', ('FIAT', 2)),
 ('4', ('CHEVROLET', 1)),
 ('6', ('FERRARI', 2))]

In [81]:
textosRDD = textosRDD.groupByKey().mapValues(list)
textosRDD.collect()

[('1', [('VW', 2), ('FORD', 1), ('FIAT', 1)]),
 ('4', [('FORD', 2), ('FIAT', 2), ('CHEVROLET', 1)]),
 ('2', [('ALFA', 2), ('ROMEO', 2), ('LANCIA', 1)]),
 ('3',
  [('FIAT', 1), ('ALFA', 1), ('ROMEO', 1), ('FERRARI', 1), ('LANCIA', 1)]),
 ('5', [('CHEVROLET', 2), ('VW', 3)]),
 ('6', [('FIAT', 1), ('FERRARI', 2)])]

In [72]:
def idf(list_doc_freq):
    return math.log(6/(len(list_doc_freq)+1))

In [82]:
textosRDD.map(lambda x: (x[0], idf(x[1]), x[1])).collect()

[('1', 0.4054651081081644, [('VW', 2), ('FORD', 1), ('FIAT', 1)]),
 ('4', 0.4054651081081644, [('FORD', 2), ('FIAT', 2), ('CHEVROLET', 1)]),
 ('2', 0.4054651081081644, [('ALFA', 2), ('ROMEO', 2), ('LANCIA', 1)]),
 ('3',
  0.0,
  [('FIAT', 1), ('ALFA', 1), ('ROMEO', 1), ('FERRARI', 1), ('LANCIA', 1)]),
 ('5', 0.6931471805599453, [('CHEVROLET', 2), ('VW', 3)]),
 ('6', 0.6931471805599453, [('FIAT', 1), ('FERRARI', 2)])]